<a href="https://colab.research.google.com/github/Shoaib1M/hackodishav1/blob/main/Audio_Classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [27]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [38]:
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
import csv

import matplotlib.pyplot as plt
from IPython.display import Audio
from scipy.io import wavfile

model = hub.load('https://tfhub.dev/google/yamnet/1')

Exception ignored in: <function Wave_write.__del__ at 0x78102763aac0>
Traceback (most recent call last):
  File "/usr/lib/python3.12/wave.py", line 465, in __del__
    self.close()
  File "/usr/lib/python3.12/wave.py", line 583, in close
    self._ensure_header_written(0)
  File "/usr/lib/python3.12/wave.py", line 606, in _ensure_header_written
    self._write_header(datasize)
  File "/usr/lib/python3.12/wave.py", line 610, in _write_header
    self._file.write(b'RIFF')
ValueError: I/O operation on closed file.
Exception ignored in: <function Wave_write.__del__ at 0x78102763aac0>
Traceback (most recent call last):
  File "/usr/lib/python3.12/wave.py", line 465, in __del__
    self.close()
  File "/usr/lib/python3.12/wave.py", line 583, in close
    self._ensure_header_written(0)
  File "/usr/lib/python3.12/wave.py", line 606, in _ensure_header_written
    self._write_header(datasize)
  File "/usr/lib/python3.12/wave.py", line 610, in _write_header
    self._file.write(b'RIFF')
ValueErr

In [39]:
def class_name_csv(class_map_csv_text):
  class_names=[]
  with tf.io.gfile.GFile(class_map_csv_text) as csvfile:
    reader=csv.DictReader(csvfile)
    for row in reader:
      class_names.append(row["display_name"])
  return class_names

class_map_path=model.class_map_path().numpy()
class_names=class_name_csv(class_map_path)

In [40]:
import scipy.signal

def ensure_sample(original_sample_rate,waveform,desired_rate=16000):
  if original_sample_rate!=desired_rate:
    desired_lenght=int(round((float(len(waveform))/original_sample_rate)*16000))
    waveform= scipy.signal.resample(waveform,desired_lenght)
  return desired_rate,waveform

In [41]:
!curl -O https://storage.googleapis.com/audioset/speech_whistling2.wav

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  153k  100  153k    0     0  1409k      0 --:--:-- --:--:-- --:--:-- 1410k


In [42]:
!curl -O https://storage.googleapis.com/audioset/miaow_16k.wav

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  210k  100  210k    0     0  2008k      0 --:--:-- --:--:-- --:--:-- 2023k


In [59]:
wav_file="/content/drive/MyDrive/test/565103__hudehel__train-station-announcement.wav"

sample_rate, waveform=wavfile.read(wav_file, mmap=False)
sample_rate, waveform=ensure_sample(sample_rate,waveform)
duration=len(waveform)/sample_rate

print(f'Sample Rate:{sample_rate} Hz')
print(f'Duration:{duration} s')
print(f'Size of input:{len(waveform)}')

from IPython.display import Audio
print("\nPlaying the audio that is about to be classified...")
waveform_flat = waveform.ravel()
int16_info = np.iinfo(np.int16)
clipped_waveform = np.clip(waveform_flat, int16_info.min, int16_info.max)
waveform_int = clipped_waveform.astype(np.int16)
Audio(waveform_int, rate=int(sample_rate))

Sample Rate:16000 Hz
Duration:29.309375 s
Size of input:468950

Playing the audio that is about to be classified...


In [60]:
waveform_float = tf.cast(waveform, tf.float32)
normal_waveform=waveform_float/tf.int16.max

In [61]:
import tensorflow as tf
waveform = tf.convert_to_tensor(waveform, dtype=tf.float32)
if len(normal_waveform.shape) > 1:
    normal_waveform = tf.reduce_mean(normal_waveform, axis=1)

score, embeddings, spectrogram=model(normal_waveform)
scores_np=score.numpy()
#int_scores=scores_np.astype(np.int32)
infered_class=class_names[scores_np.mean(axis=0).argmax()]
dic={}
t_sum=0
tt_sum=0
x=0
for i in range(len(scores_np.mean(axis=0))):
  dic[class_names[i]]=scores_np.mean(axis=0)[i]*100
sorted_dic = sorted(dic.items(), key=lambda item: item[1], reverse=True)
for key,value in sorted_dic[:10]:
  if key=="Silence" or key=="Sound effect":
    x+=1
for key,value in sorted_dic[:10+x]:
  if key!="Silence" and key!="Sound effect":
    t_sum+=value
for key,value in sorted_dic[:10+x]:
  if key!="Silence" and key!="Sound effect":
    print(f"{key}:{(value/t_sum)*100}")

Music:24.28970718383789
Speech:20.930856704711914
Alarm:16.070392608642578
Musical instrument:7.312983989715576
Inside, large room or hall:6.503000736236572
Inside, small room:6.3721208572387695
Outside, urban or manmade:4.931197166442871
Siren:4.733972549438477
Singing:4.518322467803955
Traditional music:4.337442398071289
